Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары
Если mvideo не доступен (что иногда бывает), можете использовать любой сайт с лентой специальных предложений, к примеру: https://www.eldorado.ru/, https://5ka.ru/, https://www.carrefoursa.com/ и т.п.

In [ ]:
# ! sudo docker run -d -p 27017:27017 --name m1 mongo

In [24]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [25]:
db = client['vacanciesdb']

In [26]:
def add_new_vacancy(column, name, url, min_salary, max_salary, currency):
    for x in column.find({},{'_id': 0, 'name': 1}):
        if name == x['name']:
            return False
    column.insert_one({'name': name, 'url': url, 'min_salary': min_salary, 'max_salary': max_salary, 'currency': currency})
    return True

In [27]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [28]:
def parse_hh(vacancy):
    vacancies = db[vacancy]
    params = {
        'text': vacancy,
        'search_field': 'name',
        'items_on_page': '100',
        'page': ''
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
    }
    url = 'https://hh.ru/search/vacancy'
    response = requests.get(url, params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    pages_data = dom.find('div', {'class': 'bloko-gap bloko-gap_top'}).findChild()
    last_page = int(pages_data.find_all('span')[-3].getText())
    for page in range(0, last_page):
        params['page'] = page
        response= requests.get(url, params=params, headers=headers)
        if response.ok:
            dom = bs(response.text, 'html.parser')
            vacancy_items_data = dom.find('div', {'data-qa': 'vacancy-serp__results'})
            if vacancy_items_data != None:
                vacancy_items = vacancy_items_data.find_all('div', {'class': 'serp-item'})
            for vacancy_item in vacancy_items:
                vacancy_main_data = vacancy_item.find('a', {'class': 'serp-item__title'})
                name = vacancy_main_data.getText()
                url = vacancy_main_data['href']
                vacancy_salary_data = vacancy_item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
                min_salary, max_salary, currency = None, None, None
                if vacancy_salary_data != None:
                    vacancy_salary = vacancy_salary_data.getText()
                    salary_list = re.findall('[0-9]+', vacancy_salary)
                    currency = vacancy_salary.split(' ')[-1]
                    if len(salary_list) == 2:
                        if 'от' in vacancy_salary:
                            min_salary = float(''.join(salary_list))
                        if 'до' in vacancy_salary:
                            max_salary = float(''.join(salary_list))
                    if len(salary_list) == 4:
                        min_salary = float(''.join(salary_list[:2]))
                        max_salary = float(''.join(salary_list[2:]))
                add_new_vacancy(vacancies, name, url, min_salary, max_salary, currency)

In [29]:
vacancy = 'python'
vacancy_dataframe = parse_hh(vacancy)

In [30]:
for x in db[vacancy].find()[:10]:
    print(x)

{'_id': ObjectId('63f9fe0eb1fc220c66fbc5c4'), 'name': 'Ведущий инженер по тестированию (python)', 'url': 'https://novosibirsk.hh.ru/vacancy/73199414?from=vacancy_search_list&query=python', 'min_salary': None, 'max_salary': None, 'currency': None}
{'_id': ObjectId('63f9fe0eb1fc220c66fbc5c5'), 'name': 'Разработчик Python', 'url': 'https://novosibirsk.hh.ru/vacancy/77091466?from=vacancy_search_list&query=python', 'min_salary': 200000.0, 'max_salary': 250000.0, 'currency': 'руб.'}
{'_id': ObjectId('63f9fe0eb1fc220c66fbc5c6'), 'name': 'Программист Python Junior', 'url': 'https://novosibirsk.hh.ru/vacancy/77146976?from=vacancy_search_list&query=python', 'min_salary': None, 'max_salary': None, 'currency': None}
{'_id': ObjectId('63f9fe0eb1fc220c66fbc5c7'), 'name': 'Программист Python', 'url': 'https://novosibirsk.hh.ru/vacancy/76608111?from=vacancy_search_list&query=python', 'min_salary': 45000.0, 'max_salary': None, 'currency': 'руб.'}
{'_id': ObjectId('63f9fe0eb1fc220c66fbc5c8'), 'name': 'P

In [22]:
# client.drop_database('vacanciesdb')